In [8]:
case class Parser[A](run : String => List[(A,String)]) extends AnyRef { self =>
                                                                       
    def map[B](f: A => B): Parser[B] =
        Parser { str =>
            self.run(str).map { case (value,rest) =>
                (f(value),rest)
            }
        }
    
    def filter(f: A => Boolean): Parser[A] = 
        Parser { str =>
                self.run(str).filter { case (value,_) =>
                    f(value)
                }
        }
                                                                       
    def filterMap[B](f: A => Option[B]): Parser[B] =
        Parser { str =>
                self.run(str).flatMap { case (value,rest) =>
                    f(value).map { b => (b, rest) }
                }
        }
                                                                       
    def or(other: => Parser[A]): Parser[A] = Parser { str =>
        val firstResult = self.run(str)
        if(!firstResult.isEmpty) {
            firstResult
        } else {
            other.run(str)
        }
    }
    
    def apply(str: String) = run(str)
                                                                       
    
}

val parseChar: Parser[Char] = 
    Parser { str =>
            if(str.isEmpty)
                List.empty
            else
                List( (str.head, str.tail) )
    }

val parseDigit: Parser[Int] =
    parseChar
        .filter { c => '0' <= c && c <= '9' }
        .map { c => c - '0' }

val parseTrue  = parseChar.filter( c => c == 't' ).map( _ => true )
val parseFalse = parseChar.filter( c => c == 'f' ).map( _ => false )
val parseBoolean = parseTrue or parseFalse
assert( parseBoolean("t") == List((true,"") ) )
assert( parseBoolean("f") == List((false,"") ) )
println("tests passed!")
"sfsad".substring(2)

def map2[A,B,C](p1: Parser[A], p2: => Parser[B], f: (A,B) => C): Parser[C] = Parser { str =>
    for {
        (a, rest1) <- p1.run(str)
        (b, rest2) <- p2.run(rest1)
    } yield (f(a,b), rest2)
}

case class Id(n: Int, char: Char)

def isUpperCase(char: Char): Boolean = 'A' <= char && char <= 'Z'

val parseId: Parser[Id] = 
    map2(parseDigit, parseChar.filter(isUpperCase), (n: Int, c: Char) => Id(n,c) )

parseId("1A")

def succeed[A](value: A): Parser[A] = Parser { str => List((value,str)) }

def rep[A](pa: Parser[A]): Parser[List[A]] = 
    map2(pa, rep(pa), (a:A, tl: List[A]) => a :: tl ) or succeed(List.empty)

val parseAs = rep(parseChar.filter(_ =='A'))

def repN[A](p: Parser[A], n: Int): Parser[List[A]] = 
    if(n == 0) {
        succeed(List.empty)
    } else {
        map2(p, repN(p,n-1), (head: A, tail: List[A]) => head :: tail)
    }
parseAs("AAAAxxx")
parseAs("xxx")

val twoXs = repN(parseChar.filter(_ == 'X'), 2)
twoXs("XXA")

def andThen[A,B](first: Parser[A], f: A => Parser[B]): Parser[B] = Parser { str =>
   first.run(str).map { case (a, rest1)  =>
       f(a).run(rest1)
   }.flatten
}

val parseNandReps = andThen(parseDigit,{ n: Int => repN(parseChar.filter(_ == 'X'), n) })

parseNandReps("2XX")
parseNandReps("3XX")



tests passed!


defined class Parser
parseChar: $user.Parser[Char] = Parser(<function1>)
parseDigit: $user.Parser[Int] = Parser(<function1>)
parseTrue: $user.Parser[Boolean] = Parser(<function1>)
parseFalse: $user.Parser[Boolean] = Parser(<function1>)
parseBoolean: $user.Parser[Boolean] = Parser(<function1>)
res7_9: String = "sad"
defined function map2
defined class Id
defined function isUpperCase
parseId: $user.Parser[$user.Id] = Parser(<function1>)
res7_14: List[($user.Id, String)] = List((Id(1, 'A'), ""))
defined function succeed
defined function rep
parseAs: $user.Parser[List[Char]] = Parser(<function1>)
defined function repN
res7_19: List[(List[Char], String)] = List((List('A', 'A', 'A', 'A'), "xxx"))
res7_20: List[(List[Char], String)] = List((List(), "xxx"))
twoXs: $user.Parser[List[Char]] = Parser(<function1>)
res7_22: List[(List[Char], String)] = List((List('X', 'X'), "A"))
defined function andThen
parseNandReps: $user.Parser[List[Char]] = Parser(<function1>)
res7_25: List[(List[Char], String